## 3. Convolutional Nueral Network

 ### 1) import modules

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

 ### 2) define placeholder for INPUT & LABELS

In [2]:
INPUT = tf.placeholder(tf.float32, [None, 28*28])
LABELS = tf.placeholder(tf.int32, [None])

### 3) define cnn model

<img src="images/cnn.png" alt="slp model" width=1000 align="left"/>

In [3]:
#prediction = convolutional_neural_network(input=IMAGES, output_dim=10)

####  - define convolutional_neural_network function with tf.nn.conv2d, tf.nn.max_pool, tf.nn.tanh

In [4]:
def convolutional_neural_network(input, output_dim=None):
    image = tf.reshape(input, [-1, 28, 28, 1]) #batch_size x width x height x channel

    # Conv layer1
    W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 20], stddev=0.1)) # Filter가 Weight 역활을 함
    b_conv1 = tf.Variable(tf.zeros([20]))
    h_conv1 = tf.nn.conv2d(
            image, 
            W_conv1, 
            strides=[1, 1, 1, 1], 
            padding='SAME') + b_conv1 # batch_sizex28x28x20
    fmap_conv1 = tf.nn.tanh(h_conv1)

    # Pooling(Max) layer1
    # k_size = [one_image, width, hegiht, one_channel]
    h_pool1 = tf.nn.max_pool(
        fmap_conv1, 
        ksize=[1, 2, 2, 1], 
        strides=[1, 2, 2, 1], 
        padding='SAME'
    ) # batch_sizex14x14x20

    # Conv layer2
    W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 20, 50], stddev=0.1)) # Filter가 Weight 역활을 함
    b_conv2 = tf.Variable(tf.zeros([50]))
    h_conv2 = tf.nn.conv2d(
        h_pool1, 
        W_conv2, 
        strides=[1, 1, 1, 1], 
        padding='SAME') + b_conv2 # batch_sizex14x14x50
    fmap_conv2 = tf.nn.tanh(h_conv2)

    # Pooling(Max) layer2
    h_pool2 = tf.nn.max_pool(
        fmap_conv2, 
        ksize=[1, 2, 2, 1], 
        strides=[1, 2, 2, 1], 
        padding='SAME'
    ) # batch_sizex7x7x50
    h_pool2_flat = tf.reshape(h_pool2, [-1, 50 * 7 * 7])  # batch_sizex(7x7x50)
    
    # fully-connected layer1
    W_fc1 = tf.Variable(tf.truncated_normal([50 * 7 * 7, 500], stddev=0.1))
    b_fc1 = tf.Variable(tf.zeros([500]))
    h_fc1 = tf.nn.tanh(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) # batch_sizex500

    # fully-connected layer2
    W_fc2 = tf.Variable(tf.truncated_normal([500, output_dim], stddev=0.1))
    b_fc2 = tf.Variable(tf.zeros([output_dim]))
    output = tf.matmul(h_fc1, W_fc2) + b_fc2 #batch_sizex10

    return output

prediction = convolutional_neural_network(INPUT, output_dim=10)

In [5]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=LABELS, logits=prediction
)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

### 4) load data

In [6]:
mnist = input_data.read_data_sets("./data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### 5) start training
   #### - set training parameters : batch size, learning rate, total loop

In [7]:
BATCH_SIZE = 100
LEARNING_RATE = 0.01
TOTAL_LOOP = 10000

 - arrA = [[0,0,0,0,1],[0,1,0,0,0]]
 - np.where(arrA) => ([0,1], [4,1])
 - ref) https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.where.html?highlight=numpy%20where#numpy.where

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for loop in range(1, TOTAL_LOOP + 1):
    
    train_images, train_labels = mnist.train \
                                      .next_batch(BATCH_SIZE)
    train_labels = np.where(train_labels)[1]
        
    _, loss = sess.run(
        [optimizer, cost],
        feed_dict={
            INPUT: train_images, 
            LABELS: train_labels
        }
    )
    
    if loop % 500 == 0 or loop == 0:
        print("loop: %05d,"%(loop), "loss:", loss)

print("Training Finished! (loss : " + str(loss) + ")")

loop: 00500, loss: 0.24762183
loop: 01000, loss: 0.19355531
loop: 01500, loss: 0.15968311
loop: 02000, loss: 0.13976234
loop: 02500, loss: 0.06908678
loop: 03000, loss: 0.09957447
loop: 03500, loss: 0.10375062
loop: 04000, loss: 0.05489205
loop: 04500, loss: 0.120919704
loop: 05000, loss: 0.078065485
loop: 05500, loss: 0.018431403
loop: 06000, loss: 0.03395952
loop: 06500, loss: 0.082293175
loop: 07000, loss: 0.05262528
loop: 07500, loss: 0.10878975
loop: 08000, loss: 0.071632974
loop: 08500, loss: 0.06618172
loop: 09000, loss: 0.07681943
loop: 09500, loss: 0.012724737
loop: 10000, loss: 0.023088682
Training Finished! (loss : 0.023088682)


### 6) test performance

 - test image shape: (100, 784)
 - test label shape: (100, 10) 
 

 - arrB = [[0, 1, 2],[3, 4, 5]]
 - np.argmax(arrB) => 5
 - np.argmax(arrB, axis=0) => [1, 1, 1]
 - np.argmax(arrB, axis=1) => [2, 2]
 - ref) https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.argmax.html
 

In [9]:
TEST_SAMPLE_SIZE = 100
TEST_NUMBER = 5
accuracy_save = dict()

for number in range(1, 1+TEST_NUMBER):
    
    test_images, test_labels = mnist.test \
                                    .next_batch(TEST_SAMPLE_SIZE)
    pred_result = sess.run(
        prediction, 
        feed_dict={INPUT: test_images}
    )

    pred_number = np.argmax(pred_result, axis=1) # 100x1
    label_number = np.where(test_labels)[1] #100x1
    
    accuracy_save[number] = np.sum(pred_number == label_number)
    
print("Accuracy:", accuracy_save)
print("Total mean Accuracy:", 
      np.mean(list(accuracy_save.values()))
)
      

Accuracy: {1: 95, 2: 100, 3: 99, 4: 99, 5: 99}
Total mean Accuracy: 98.4
